# Prepping Reptilia Data for Inputting into R Script 

In [23]:
import pandas as pd

In [24]:
rep = pd.read_csv('reptilia_dup.csv')

In [25]:
# Cursory glance
rep.head()

,occurrence_no,pres_mode,preservation_quality,common_body_parts,abund_value,abund_unit,class,family,genus,taxon_environment,...,artifacts,component_comments,spatial_resolution,temporal_resolution,lagerstatten,concentration,orientation,abund_in_sediment,collection_comments,taxonomy_comments
0,N84,NaN,NaN,an individual seprated from the 11753 b and c ...,NaN,NaN,Reptilia,Rhynchosauridae,Stenaulorhynchus,terrestrial,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,N83,NaN,NaN,lower jaw and humerus,NaN,NaN,Reptilia,Rhynchosauridae,Mesosuchus,terrestrial,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,N82,NaN,NaN,foot,NaN,NaN,Reptilia,Rhynchosauridae,Mesosuchus,terrestrial,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,N81,NaN,NaN,maxilla,NaN,NaN,Reptilia,Rhynchosauridae,Mesosuchus,terrestrial,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,N80,NaN,NaN,partial skull,NaN,NaN,Reptilia,Rhynchosauridae,Eohyosaurus,terrestrial,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
rep.columns

Index(['occurrence_no', 'pres_mode', 'preservation_quality',
       'common_body_parts', 'abund_value', 'abund_unit', 'class', 'family',
       'genus', 'taxon_environment', 'life_habit', 'diet', 'early_interval',
       'late_interval', 'time_bins', 'museum', 'collection_no', 'age_max',
       'age_min', 'age_median', 'age_uncer_range', 'lng', 'lat', 'environment',
       'occurrence_comments', 'cc', 'state', 'county', 'formation',
       'stratgroup', 'member', 'zone', 'accepted_rank', 'ref_author',
       'ref_pubyr', 'reference_no.x', 'collection_name', 'collection_subset',
       'collection_aka', 'latlng_basis', 'latlng_precision', 'geogscale',
       'geogcomments', 'paleomodel', 'geoplate', 'paleoage', 'paleolng',
       'paleolat', 'paleomodel2', 'geoplate2', 'paleoage2', 'paleolng2',
       'paleolat2', 'paleomodel3', 'geoplate3', 'paleoage3', 'paleolng3',
       'paleolat3', 'protected', 'stratscale', 'localsection', 'localbed',
       'stratcomments', 'lithdescript', 'litho

### Choosing an "Area" Column

'collection_no" will definitely be "Locality" column in the final table

'genus' will definitely be "Taxon" column in the final table

**Now I need to choose a 'Area' column**


In [27]:
# Looking for Nulls in the potential "Area" columns
nulls_zone = rep[rep['zone'].isnull()]
nulls_cc = rep[rep['cc'].isnull()]
len(nulls_zone), len(nulls_cc)


(3545, 8)

In [28]:
# Seeing how many unique values are in the potential "Area" columns
len(rep['zone'].unique()), len(rep['cc'].unique())


(107, 52)

'zone' column has more nulls and more unique values, so I'm going to go for the 'cc' column as the 'Area' column in the final dataset

In [29]:
# Filtering to just the columns deepdive needs
rep_filtered = rep[['genus', 'collection_no', 'age_max', 'age_min', 'cc']]
rep_filtered = rep_filtered.rename(columns={'genus': 'Taxon', 'collection_no': 'Locality', 'cc': 'Area', 'age_max':'MaxAge', 'age_min':'MinAge'})
rep_filtered.head()

,Taxon,Locality,MaxAge,MinAge,Area
0,Stenaulorhynchus,11753,247.2,242.0,TZ
1,Mesosuchus,7838,247.2,242.0,ZA
2,Mesosuchus,6546,247.2,242.0,ZA
3,Mesosuchus,5887,247.2,242.0,ZA
4,Eohyosaurus,K11831,247.2,242.0,ZA


### Imputing any Nulls


In [33]:
# Only 'Area' has nulls
rep_filtered.isna().sum()

Taxon       0
Locality    0
MaxAge      0
MinAge      0
Area        8
dtype: int64

In [35]:
rep_filtered[rep_filtered['Area'].isna()]

,Taxon,Locality,MaxAge,MinAge,Area
2762,Mesosaurus,90780,290.1,283.5,NaN
2763,Mesosaurus,90755,290.1,283.5,NaN
2764,Mesosaurus,90755,290.1,283.5,NaN
3721,Plateosaurus,64316,208.5,201.4,NaN
4259,Mesosaurus,28471,290.1,283.5,NaN
4260,Mesosaurus,28470,290.1,283.5,NaN
4261,Mesosaurus,28469,290.1,283.5,NaN
4262,Mesosaurus,28468,290.1,283.5,NaN


In [41]:
# Seeing if the locality with nulls in 'Area' has any other entries. 
# If it does, we can infer the 'Area' with info from the other entries
null1 = rep[rep['collection_no'] == "90780"]
null2 = rep[rep['collection_no'] == "90755"]
null3 = rep[rep['collection_no'] == "64316"]
null4 = rep[rep['collection_no'] == "28471"]
null5 = rep[rep['collection_no'] == "28470"]
null6 = rep[rep['collection_no'] == "28469"]
null7 = rep[rep['collection_no'] == "28468"]

In [43]:
# Seeing length of null entries for those localities
len(null1), len(null2), len(null3), len(null4), len(null5), len(null6), len(null7)
# Only null2 has >1

(1, 2, 1, 1, 1, 1, 1)

In [46]:
null2[['cc', 'state', 'county', 'formation', 'stratgroup', 'member', 'zone', 'collection_no']]


,cc,state,county,formation,stratgroup,member,zone,collection_no
2763,NaN,Karas,NaN,Whitehill,Ecca,NaN,NaN,90755
2764,NaN,Karas,NaN,Whitehill,Ecca,NaN,NaN,90755


In [50]:
rep[rep['cc'] == 'Karas']

,occurrence_no,pres_mode,preservation_quality,common_body_parts,abund_value,abund_unit,class,family,genus,taxon_environment,...,artifacts,component_comments,spatial_resolution,temporal_resolution,lagerstatten,concentration,orientation,abund_in_sediment,collection_comments,taxonomy_comments


In [51]:
# There are no reasonable values for these nulls to be imputed with, so I'll just drop them

# Checking original length of filtered df
rep_filtered.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4411 entries, 0 to 4410
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Taxon     4411 non-null   object 
 1   Locality  4411 non-null   object 
 2   MaxAge    4411 non-null   float64
 3   MinAge    4411 non-null   float64
 4   Area      4403 non-null   object 
dtypes: float64(2), object(3)
memory usage: 172.4+ KB


In [52]:
# Dropping nulls, new df's length is correct
rep_filtered_dropped = rep_filtered.dropna()
rep_filtered_dropped.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4403 entries, 0 to 4410
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Taxon     4403 non-null   object 
 1   Locality  4403 non-null   object 
 2   MaxAge    4403 non-null   float64
 3   MinAge    4403 non-null   float64
 4   Area      4403 non-null   object 
dtypes: float64(2), object(3)
memory usage: 206.4+ KB


In [10]:
single_area = value_counts[value_counts == 1]
len(single_area)

32

In [11]:
single_area.head(32)

Area
Beneckeia buchi                                   1
Cynognathus subzone B                             1
Gymnotoceras blakei                               1
Daptocephalus Assemblage Zone                     1
pulcher/robustus                                  1
enodis-laevigatus                                 1
Lootsbergian                                      1
Daptocephalus Assemblage                          1
Cistecephalus                                     1
Trachyceras aon                                   1
evolutus                                          1
Nevadisculites taylori                            1
Rotelliformis/Meeki                               1
MassetognathusâChanaresuchus                    1
Eoprotrachyceras curionii                         1
Psiloceras johnstoni                              1
Dinodontosaurus AZ                                1
Frechites occidentalis                            1
aonoides                                          1
Prohung

In [18]:
rep_filtered.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4411 entries, 0 to 4410
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Taxon     4411 non-null   object 
 1   Locality  4411 non-null   object 
 2   MaxAge    4411 non-null   float64
 3   MinAge    4411 non-null   float64
 4   Area      866 non-null    object 
dtypes: float64(2), object(3)
memory usage: 172.4+ KB


In [13]:

# Confirmed that in the CSV the "NaN"'s show up as empty cells, not as "NaN"

,Taxon,Locality,MaxAge,MinAge,Area
166,Garzapelta,79947,227.000,208.5,NaN
167,Marcianosuchus,134304,247.200,244.7,NaN
168,Unguinychus,235534,227.000,208.5,NaN
170,Mixosaurus,235296,247.200,242.0,NaN
171,Idiosaura,39159,237.000,227.0,NaN
...,...,...,...,...,...
4406,Procolophon,13043,251.902,247.2,NaN
4407,Kuehneosaurus,12858,208.500,201.4,NaN
4408,Kuehneosuchus,12857,208.500,201.4,NaN
4409,Rutiodon,12856,227.000,208.5,NaN


In [15]:
# Looking for Empty values
empty_values = rep_filtered[rep_filtered['Area'].str.strip() == '']
empty_values

,Taxon,Locality,MaxAge,MinAge,Area


### 'Locality' EDA

In [11]:
# Finding number of unique collection sites
len(rep_filtered['Locality'].unique())

1733

In [16]:
# Finding all collection sites with only one instance (singletons)
singletons = rep_filtered['Locality'].value_counts()
singletons = singletons[singletons == 1]
len(singletons)

1098

In [20]:
# Finding number of unique species
rep_filtered['Taxon'].value_counts()

Taxon
Hovasaurus        301
Captorhinus       127
Nothosaurus       111
Grallator         108
Plateosaurus       99
                 ... 
Sophineta           1
Czatkowiella        1
Sarcosaurus         1
Melanorosaurus      1
Vonhuenia           1
Name: count, Length: 700, dtype: int64

In [22]:
# Finding number of unique Areas
rep_filtered['Area'].value_counts()

Area
Hyperodapedon                    91
Perovkan                         61
Dinodontosaurus                  34
Ictidosauria                     29
Cynognathus subzone C            27
                                 ..
Lootsbergian                      1
pulcher/robustus                  1
Daptocephalus Assemblage Zone     1
Daptocephalus Assemblage          1
IV                                1
Name: count, Length: 106, dtype: int64

In [23]:
# Saving it off for now
rep_filtered.to_csv('reptilia_filtered.csv', index=False)